In [3]:
import polars as pl
import json

In [ ]:
def longest_common_subsequence(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    return dp[m][n]

def find_largest_common_subsequence(string, string_list):
    """
    Given a string and a list of strings, returns the string
    in the list with the largest common subsequence with the input string.

    Args:
    - string (str): The input string.
    - string_list (list): List of strings to find the largest common subsequence.

    Returns:
    - str or None: The string with the largest common subsequence, or None if the list is empty.
    """
    max_lcs_length = 0
    result_string = None

    for candidate_string in string_list:
        lcs_length = longest_common_subsequence(string, candidate_string)
        if lcs_length > max_lcs_length:
            max_lcs_length = lcs_length
            result_string = candidate_string

    return result_string

In [26]:
data = pl.scan_csv('C:/Users/aurel/Downloads/HS23/CIR/Automatic-Cart/en.openfoodfacts.org.products.csv', separator='\t')
to_keep = ['product_name', 'nutriscore_score', 'image_small_url', 'energy-kcal_100g', 'fat_100g','carbohydrates_100g','proteins_100g']

with open('C:/Users/aurel/Downloads/HS23/CIR/Automatic-Cart/ImageToGraph/supermarket_items.json') as f:
    items = json.load(f)

items_list =  []
for i,product in enumerate(items.values()):
    if i != 6: #Skip the tools
        items_list.extend(product)

transformed_dict = {}
for category, items in items.items():
    for item in items:
        transformed_dict[item] = category

data = data.select(to_keep)
data = data.filter(pl.col('product_name').str.contains('|'.join(items_list)))
data = data.filter(pl.col("proteins_100g").is_not_null() &  pl.col("nutriscore_score").is_not_null() &
                   pl.col("image_small_url").is_not_null() &  pl.col("energy-kcal_100g").is_not_null() &
                   pl.col("fat_100g").is_not_null() &  pl.col("carbohydrates_100g").is_not_null())
data = data.with_columns(
    (
        pl.col('product_name').map_elements(
            lambda x: find_largest_common_subsequence(x, items_list)
        )
    ).alias('product_name')
)
data = data.with_columns(pl.col('product_name').map_dict(transformed_dict).alias('category'))
data = data.unique('product_name')
data = data.collect()
data.write_csv('Food_Data.csv')

PanicException: python function failed KeyboardInterrupt: 

PanicException: python function failed KeyboardInterrupt: 

In [1]:
from sklearn.feature_selection import SelectPercentile, mutual_info_classif
import pandas as pd

# Assuming 'train' is your DataFrame and 'state' is the target column
# You may need to replace this with your actual column names and DataFrame
train = pd.DataFrame({
    'feature1': [1, 2, 3, 4],
    'feature2': [5, 6, 7, 8],
    'feature3': [9, 10, 11, 12],
    'state': [0, 1, 0, 1]
})

# Drop the target column for feature selection
X = train.drop('state', axis=1)
y = train['state']

# SelectPercentile with mutual_info_classif
sel = SelectPercentile(mutual_info_classif, percentile=70)
sel.fit(X, y)

# Get the mask of selected features
selected_features_mask = sel.get_support()

# Get the names of the dropped features
dropped_features = X.columns[~selected_features_mask]

print("Dropped Features:", dropped_features.tolist())

Dropped Features: ['feature3']
